In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import time
import random



def check_data(data):
    try:
        technicalSheet = data['props']['pageProps']['data']['technicalSheet']
        longitude = data['props']['pageProps']['data']['longitude']
        latitude = data['props']['pageProps']['data']['latitude']
        stratum = technicalSheet[0]['value'] if technicalSheet[0]['value'] is not None  else 0
        bathrooms = technicalSheet[3]['value'] if technicalSheet[3]['value'] is not None  else 0
        constructed_area = float(technicalSheet[4]['value'].replace('  m2', '')) if technicalSheet[4]['value'] is not None  else 0
        house_age = technicalSheet[6]['value'] 
        bedrooms = technicalSheet[7]['value'] if technicalSheet[7]['value'] is not None  else 0
        parking = technicalSheet[8]['value'] if technicalSheet[8]['value'] is not None  else 0
        administration = float(technicalSheet[9]['value'].replace(',', '')) if technicalSheet[9]['value'] is not None  else 0
        floor = technicalSheet[10]['value'] if technicalSheet[10]['value'] is not None  else 0
        rent_price = data['props']['pageProps']['data']['price']['amount'] 
        rent_price = rent_price if rent_price is not None else 0

        if house_age is not None and house_age != 'menor a 1 año' and house_age != 'más de 30 años':
            age_range = house_age.replace(' años', '').split(' a ')
            age_range = list(map(int, age_range))
            house_age = int(sum(age_range)/len(age_range))
        else: 
            if house_age == 'más de 30 años':
                house_age = 31
            else:
                house_age = 0
        
        return [longitude, latitude, stratum, bathrooms, constructed_area, house_age, bedrooms, parking, administration, floor, rent_price]
    
    except KeyError as e:
        print(f'Error: clave {e} no encontrada en los datos JSON.')
        return [None]*11

In [ ]:
df = pd.DataFrame(columns=['longitude', 'latitude', 'stratum', 'bathrooms', 'constructed_area', 'house_age', 'bedrooms', 'parking', 'administration', 'floor', 'rent_price'])
n_pages = 200
next_page = ''

headers = {
    # Encabezado User-Agent que indica: navegador compatible con Mozilla, en Windows 10 (64-bit), usando el motor 
    # WebKit como Chrome v91, con referencias a Gecko y Safari para compatibilidad
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

session = requests.Session()
# itera sobre las 200 primeras paginas de apartamentos en arriendo en finca raiz
for i in range(1, n_pages+1):
    # Se usa try para prevenir errores y no suspenda la consulta
    try:
        url = 'https://www.fincaraiz.com.co/arriendo/apartamentos/bogota/bogota-dc' + next_page
        html_text = session.get(url, headers=headers).text
    except requests.exceptions.RequestException as e:
        print(f'Error al conectar con la pagina {url}\n {e}')
        continue
    
    #la siguiente linea pausa la ejecucion del codigo durante un tiempo aleatorio entre 1 y 4 seg para evitar que la pagina bloquee las consultas
    time.sleep(random.uniform(1, 4))
    #El archivo html se convierte a archivo lxml que una estructura mas eficiente para buscar en el DOM
    soup1 = BeautifulSoup(html_text, 'lxml')
    #Esta linea busca todos los links con la clase 'lc-cardCover' que son las cards de los apartamentos
    cards = soup1.find_all('a', class_ = 'lc-cardCover')

    #Se itera sobre todas las cards obtenidas y se extrae el atributo href para hacer la consulta al apartamento especifico
    for j in cards:
        try:
            url_apartment_features = 'https://www.fincaraiz.com.co'+j.get('href')
            apartment_features = session.get(url_apartment_features, headers=headers).text
            soup_apartment_f = BeautifulSoup(apartment_features, 'lxml')
            script_data = soup_apartment_f.find('script', id='__NEXT_DATA__')
            if script_data:
                #Se convierte script_data de JSON a un diccionario de Python para acceder facilmente a los datos
                json_data = json.loads(script_data.string)
                #Se extraen los features con check_data y se agregan al dataframe
                df.loc[len(df)] = check_data(json_data)
            else:
                print(f"No se encontró el script en {url_apartment_features}")
            #Nuevamente se demora la consulta para evitar bloqueos
            time.sleep(random.uniform(1,4))
        except requests.exceptions.RequestException as e:
            print(f'Error al conectar con la card {url_apartment_features}\n {e}')
            continue
    #Cuando se consultan todas las cards se cambia de pagina agregando 'pagina' y el consecutivo numero
    next_page = 'pagina' + str(i+1)

df.to_csv('Real_State_Bogota1.csv', index=False)
print(df)

No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191367699
Error: clave 'data' no encontrada en los datos JSON.
Error: clave 'data' no encontrada en los datos JSON.
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191390561
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191292420
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/11105019
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191385799
Error: clave 'data' no encontrada en los datos JSON.
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191328174
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191462974
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/10504098
No se encontró el script en https://www.fincaraiz.com.co/apartamento-en-arriendo/191249123
Error: clave 'data' no e

: 

In [24]:
data = pd.read_csv('Real_State_Bogota.csv')
data.describe()

,longitude,latitude,stratum,bathrooms,constructed_area,house_age,bedrooms,parking,administration,floor
count,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1229.000000,1.229000e+03,1229.000000
mean,-74.101855,4.723894,4.965826,2.481692,123.599487,15.004882,2.393816,1.533767,5.420826e+05,3.401953
std,0.255839,0.433605,1.216929,1.167850,95.469204,11.255472,0.935803,1.147100,8.105327e+05,3.817864
min,-76.533379,3.354820,0.000000,0.000000,17.260000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,-74.068681,4.659539,4.000000,2.000000,57.000000,4.000000,2.000000,1.000000,0.000000e+00,0.000000
50%,-74.052639,4.681236,5.000000,2.000000,90.000000,12.000000,2.000000,1.000000,5.000000e+04,3.000000
75%,-74.045067,4.702231,6.000000,3.000000,162.000000,23.000000,3.000000,2.000000,8.314000e+05,5.000000
max,-74.020894,11.206998,6.000000,6.000000,650.000000,31.000000,6.000000,8.000000,5.667000e+06,26.000000


: 